# CMSTP UAC Bypass via COM Object Access
Detects UAC Bypass Attempt Using Microsoft Connection Manager Profile Installer Autoelevate-capable COM Objects

## Rule Content
```
- title: CMSTP UAC Bypass via COM Object Access
  id: 4b60e6f2-bf39-47b4-b4ea-398e33cfe253
  status: stable
  description: Detects UAC Bypass Attempt Using Microsoft Connection Manager Profile
    Installer Autoelevate-capable COM Objects
  tags:
  - attack.defense_evasion
  - attack.privilege_escalation
  - attack.execution
  - attack.t1088
  - attack.t1191
  - attack.g0069
  - car.2019-04-001
  author: Nik Seetharaman
  modified: 2019/07/31
  references:
  - http://www.endurant.io/cmstp/detecting-cmstp-enabled-code-execution-and-uac-bypass-with-sysmon/
  - https://twitter.com/hFireF0X/status/897640081053364225
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      ParentCommandLine: '*\DllHost.exe'
    selection2:
      ParentCommandLine:
      - '*{3E5FC7F9-9A51-4367-9063-A120244FBEC7}'
      - '*{3E000D72-A845-4CD9-BD83-80C07C3B881F}'
    condition: selection1 and selection2
  fields:
  - CommandLine
  - ParentCommandLine
  - Hashes
  falsepositives:
  - Legitimate CMSTP use (unlikely in modern enterprise environments)
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_command_line.keyword:*\\DllHost.exe AND process_parent_command_line.keyword:(*\{3E5FC7F9\-9A51\-4367\-9063\-A120244FBEC7\} OR *\{3E000D72\-A845\-4CD9\-BD83\-80C07C3B881F\}))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()